# Working details of YOLO

You Only Look Once (YOLO) et ses variantes sont l'un des principaux algorithmes de détection d'objets. Dans cette section, nous comprendrons à un niveau élevé comment YOLO fonctionne et les limitations potentielles des cadres de détection d'objets basés sur R-CNN que YOLO surmonte.

Tout d'abord, découvrons les limitations possibles des algorithmes de détection basés sur R-CNN. .Cependant, dans la couche entièrement connectée, où seule la sortie de pooling RoI de la région détectée est transmise en entrée, dans le cas de régions qui n'englobent pas complètement l'objet (où l'objet est au-delà des limites de la zone de délimitation de la proposition de région), le réseau doit deviner les vraies limites de l'objet, car il n'a pas vu l'image complète (mais n'a vu que la proposition de région).

YOLO est pratique dans de tels scénarios, car il examine l'ensemble de l'image tout en prédisant le cadre de délimitation correspondant à une image.

De plus, Faster R-CNN est toujours lent, car nous avons deux réseaux : le RPN et le réseau final qui prédit les classes et les cadres de délimitation autour des objets.

Ici, nous comprendrons comment YOLO surmonte les limites de Faster R-CNN, à la fois en regardant l'image entière à la fois et en ayant un seul réseau pour faire des prédictions. Nous verrons comment les données sont préparées pour YOLO à travers l'exemple suivant:

* **1** Créer un exemple pour entraîner un modèle pour une image donnée:
  * Considérons une image avec les donnée des boîtes englobantes suivante:

  * Divisez l'image en N x N cellules de grille - pour l'instant, disons N=3
<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRPY9X-JzrhJWqnRK8CwTWkIdTE4loNz2Rw7zWy2A0AHvUn-PQkE1z4aqYFzm4uFzakDBE&usqp=CAU' width=500px>

* Identifiez les cellules de la grille qui contiennent le centre d'au moins une boîte englobante de vérité . Dans notre cas, ce sont les cellules (3,2) et (2,2) de notre image 3 x 3

* La ou les cellules où tombe le point médian de la boîte englobante de vérité  sont responsables de la prédiction de la boîte englobante de l'objet. Créons la vérité correspondant à chaque cellule.

* La vérité terrain de sortie correspondant à chaque cellule est la suivante:

<img src='http://media5.datahacker.rs/2019/10/yolo-anchor-boxes-1024x444.png' width=700px>


 * Ici, pc (le score d'objectivité) est la probabilité que la cellule contienne un objet.

 Comprenons comment calculer bx, by, bw et bh.

 Tout d'abord, nous considérons la cellule de grille (considérons la cellule de grille (2,2) comme notre univers, et la normalisons à une échelle entre 0 et 1

 bx et by sont les emplacements du point centre de la boîte de délimitation de la vérité par rapport à l'image (de la cellule de la grille), comme défini précédemment. Dans notre cas, bx = 0,5, car le point centre de la vérité est à une distance de 0,5 unité de l'origine. De même, by= 0,5.

Jusqu'à présent, nous avons calculé les décalages entre le centre de la cellule de la grille et le centre de vérité  correspondant à l'objet dans l'image. Maintenant, comprenons comment bw et bh sont calculés.

bw est le rapport de la largeur de la boîte englobante par rapport à la largeur de la cellule de la grille

bh est le rapport de la hauteur de la boîte englobante par rapport à la hauteur de la cellule de la grille.

Ensuite, nous allons prédire la classe correspondant à la cellule de la grille. Si nous avons trois classes (c1 – camion, c2 – voiture, c3 – bus), nous prédirons la probabilité de la cellule contenant un objet parmi l'une des trois classes. Notez que nous n'avons pas besoin d'une classe d'arrière-plan ici, car pc correspond à si la cellule de la grille contient un objet.

Maintenant que nous comprenons comment représenter la couche de sortie de chaque cellule, comprenons comment nous construisons la sortie de notre cellule de grille 3 x 3

Considérons la sortie ded cellules de grille


<img src='https://miro.medium.com/max/1400/0*5Pembl6rElHAAUci.jpg' width=700px>




La sortie de la cellule (0,0) est comme indiqué dans la capture d'écran précédente. Comme la cellule de la grille ne contient pas d'objet, la première sortie (pc - object score) est 0 et les valeurs restantes n'ont pas d'importance car la cellule ne contient pas le centre d'aucune boîte englobante de vérité terrain d'un objet



La sortie précédente est telle qu'elle est car la cellule de la grille contient un objet avec les valeurs bx, by, bw et bh qui ont été obtenues de la même manière que nous l'avons vu plus tôt (dans la puce avant-dernière), et enfin la classe étant voiture résultant en c2 étant 1 tandis que c1 et c3 sont 0


Notez que pour chaque cellule, nous sommes capables de récupérer 8 sorties. Par conséquent, pour la grille 3 x3 de cellules, nous récupérons 3 x 3 x 8 sorties

* **2** Définissez un modèle où l'entrée est une image et la sortie est de 3 x 3 x 8 avec la vérité terrain telle que définie à l'étape précédente

B = 3*3
<img src='https://www.researchgate.net/profile/Thi-Le-5/publication/337705605/figure/fig3/AS:831927326089217@1575358339500/Structure-of-one-output-cell-in-YOLO.ppm' width=700px>





 * **3** Définir la vérité en considérant les boîtes d'ancrage

 Jusqu'à présent, nous avons conçu un scénario dans lequel on s'attend à ce qu'il n'y ait qu'un seul objet dans une cellule de grille. Cependant, en réalité, il peut y avoir des scénarios où il y a plusieurs objets dans la même cellule de grille. Cela entraînerait la création de vérités fondamentales incorrectes. Comprenons ce phénomène à travers l'image d'exemple suivante:

 <img src='https://www.dlology.com/static/media/uploads/yolo/car-person.jpg' width=400px>

Dans l'exemple précédent, le point centre des cadres de délimitation de la vérité  pour la voiture et la personne tombe dans la même cellule - cellule (2,2)

Une façon d'éviter un tel scénario est d'avoir une grille qui a plus de lignes et de colonnes - par exemple, une grille 19 x 19. Cependant, il peut toujours y avoir un scénario où une augmentation du nombre de cellules de la grille n'aide pas. Les boîtes d'ancrage sont utiles dans un tel scénario. Disons que nous avons deux boîtes d'ancrage - une qui a une hauteur supérieure à la largeur (correspondant à la personne) et une autre qui a une largeur supérieure à la hauteur (correspondant à la voiture)

En règle générale, les boîtes d'ancrage auraient le centre des cellules de la grille comme centres. La sortie pour chaque cellule dans un scénario où nous avons deux boîtes d'ancrage est représentée comme une concaténation de la sortie attendue des deux boîtes d'ancrage.



 <img src='http://media5.datahacker.rs/2018/11/anchor2.png' width=700px>

 

Ici, $b_x$, $b_y$, $b_w$ et $b_h$ représentent le décalage par rapport à la boîte d'ancrage (qui est l'univers dans ce scénario comme on le voit dans l'image au lieu de la cellule de la grille).

À partir de la capture d'écran précédente, nous voyons que nous avons une sortie de 3 x 3 x 16, car nous avons deux ancres. La sortie attendue est de la forme N x N x(num_classes + 1) x (num_anchor_boxes), où N x N est le nombre de cellules dans la grille, num_classes est le nombre de classes dans l'ensemble de données et num_anchor_boxes est le nombre de boîtes d'ancrage.

* **4** Nous définissons maintenant la fonction de perte pour entraîner le modèle.

Lors du calcul de la perte associée au modèle, nous devons nous assurer de ne pas calculer la perte de régression et la perte de classification lorsque le score d'objectivité est inférieur à un certain seuil (cela correspond aux cellules qui ne contiennent pas d'objet)

Ensuite, si la cellule contient un objet, nous devons nous assurer que la classification entre les différentes classes est aussi précise que possible

Enfin, si la cellule contient un objet, les décalages du cadre de délimitation doivent être aussi proches que possible de l'attendu. Cependant, étant donné que les décalages de largeur et de hauteur peuvent être beaucoup plus élevés par rapport au décalage du centre (comme les décalages du centre sont compris entre 0 et 1, alors que les décalages de largeur et de hauteur n'ont pas besoin), nous donnons un poids inférieur aux décalages de largeur et la hauteur en récupérant une valeur de racine carrée


Calculez la perte de localisation et de classification comme suit:

$$L_{loc} = \lambda_{coord}\sum_{i=0}^{s^2}\sum_{j=0}^B1_{ij}^{obj}[(x_i-\hat{x_i})^2+(y_i-\hat{y_i})^2+(\sqrt{w_i}-\sqrt{\hat{w_i}})^2+(\sqrt{h_i}-\sqrt{\hat{h_i}})^2]$$


$$L_{cls} = \sum_{i=0}^{s^2}\sum_{j=0}^B(1_{ij}^{obj}+\lambda_{noobj}(1-1_{ij}^{obj}))(C_{ij}-\hat{C_{ij}})^2 + \sum_{i=0}^{s^2}\sum_{c\in C}1_{i}^{obj}(p_i(c)-\hat{p_i}(c))^2 $$

$$L = L_{loc}+L_{cls}$$

Ici, nous observons ce qui suit:

* $\lambda_{coord}: est le poids associé à la perte de régression
* $1_{ij}^{obj}$ représente si la cellule contient un objet

*$\hat{p_i}(c)$ : 
correspond à la probabilité de classe prédite et représente le score d'objectivité

La perte globale est une somme des valeurs de perte de classification et de régression.

Avec cela en place, nous sommes maintenant en mesure d'entraîner un modèle pour prédire les boîtes englobantes autour des objets. Cependant, pour une meilleure compréhension de YOLO et de ses variantes, nous vous encourageons à parcourir les articles originaux. Maintenant que nous comprenons comment YOLO prédit les cadres de délimitation et la classe d'objets en un seul plan.
